In [50]:
import re
import pandas as pd
import os

def lm_emulator(prompt):
    test_text = """
        Ich werde die bereitgestellten Daten analysieren und eine Einschätzung zur Wirkung der Impfkampagne auf die Infektionsrate in NRW im Januar 2022 vornehmen.

        1. Analyse der aktuellen Situation:
        Infektionsrate: 0.5
        Fallzahlen: 1000
        Demografische Impftrends: Frauen werden häufiger geimpft als Männer. Menschen mit Hypertonie haben eine höhere Impfquote, besonders über 60-Jährige.
        Regionale Impfquote: NRW hat die höchste Impfquote unter den Bundesländern.
        Bevölkerungsstruktur: NRW hat eine hohe Bevölkerungsdichte, was die Virusübertragung begünstigt.
        Impfquote nach Alter: Ältere Menschen, besonders über 60, werden häufiger geimpft, was den Schutz der vulnerablen Gruppen verbessert.
        2. Wissenschaftliche Einschätzung der Impfkampagnenwirkung:
        Impfkampagnen führen kurzfristig zu einer Reduktion der Infektionsrate, da mehr Menschen immunisiert werden, was die Übertragung reduziert.
        Herdenimmunitätseffekt: Da die Impfquote unter älteren Menschen hoch ist, wird erwartet, dass schwere Krankheitsverläufe reduziert werden.
        Geschlechtsspezifischer Einfluss: Männer haben eine niedrigere Impfquote, was potenziell zu einer höheren Infektionsrate in dieser Gruppe führen könnte.
        Einfluss der Privatversicherung: Da Männer tendenziell häufiger privat versichert sind, könnte eine unterschiedliche Impfpriorisierung eine Rolle spielen.
        3. Prognose für die Infektionsrate in NRW nach der Impfkampagne:
        Die Impfkampagne wird voraussichtlich die Infektionsrate senken, jedoch nicht sofort drastisch, da Impfungen einige Zeit benötigen, um eine Immunreaktion zu erzeugen. Basierend auf bisherigen Impfkampagnen und der aktuellen Ausgangslage, könnte die Infektionsrate um 10–20% gesenkt werden.

        Geschätzte Infektionsrate nach der Impfkampagne:
        Infektionsrate: [0.4]
    """
    return test_text

In [51]:

# Define cache file
class Cache:
    def __init__(self, columns=None):
        """
        Initialize the cache with customizable columns.
        Default columns: ['key1', 'key2', 'key3', 'value']
        """
        self.CACHE_FILE = "cache.csv"
        self.columns = columns if columns else ["key1", "key2", "key3", "value"]
        self.key_columns = self.columns[:3]  # First three columns as keys
        self.initialize_cache()

    def initialize_cache(self):
        """Ensure the cache file exists with the correct columns."""
        if not os.path.exists(self.CACHE_FILE):
            df = pd.DataFrame(columns=self.columns)
            df.to_csv(self.CACHE_FILE, index=False)

    def read_cache(self):
        """Read cache from CSV."""
        return pd.read_csv(self.CACHE_FILE)

    def add_to_cache(self, *values):
        """Add or update a row in the cache based on the first three columns as keys."""
        df = self.read_cache()
        new_entry = dict(zip(self.columns, values))

        # Check if row with matching key columns exists
        mask = df[self.key_columns].apply(tuple, axis=1) == tuple(values[:3])
        if mask.any():
            df.loc[mask, self.columns[3:]] = values[3:]
        else:
            df = pd.concat([df, pd.DataFrame([new_entry])], ignore_index=True)

        df.to_csv(self.CACHE_FILE, index=False)

    def get_from_cache(self, *keys):
        """Retrieve a value from the cache based on the first three columns as keys."""
        df = self.read_cache()

        mask = df[self.key_columns].apply(tuple, axis=1) == tuple(keys)
        result = df[mask]
        return result.iloc[0].to_dict() if not result.empty else None

In [52]:
def extract_between_brackets(text):
    """
    Extracts and returns the content inside square brackets from a string.
    If multiple brackets exist, it returns a list of all matches.

    :param text: The input string.
    :return: A string if one match, a list if multiple matches, or None if no match.
    """
    matches = re.findall(r'\[(.*?)\]', text)  # Extract content inside square brackets
    if not matches:
        return None  # Return None if no matches found
    return matches if len(matches) > 1 else matches[0]  # Return list if multiple, string if one


In [53]:
def parse_lm_output(output):
    """
    Parses the output from the language model and extracts the predicted values.

    :param output: The output text from the language model.
    :return: The predicted value(s) as a string or list of strings.
    """

    rate_str = extract_between_brackets(output)
    if rate_str is not None:
        if type(rate_str) != list:
            try:
                return float(rate_str)
            except ValueError:
                return None
        else:
            try:
                return float(rate_str[0])
            except ValueError:
                return None
    return

In [54]:
def generate_prompt(bundesland, jahr, monat, fallzahlen, infektionsrate):
    general_prompt = f"Wir führen eine Impfkampagne durch, um die Infektionsrate in {bundesland} im Jahr {jahr} und Monat {monat} zu senken. Analysiere die bereitgestellten Daten zur aktuellen Impfquote, Infektionsrate und weiteren relevanten Faktoren. Berücksichtige dabei dein Wissen über die Region, einschließlich Demografie, Gesundheitsinfrastruktur und bisheriger Impfeffekte. Schätze ein, wie sich die Impfkampagne voraussichtlich auf die Infektionsrate auswirken wird, und begründe deine Einschätzung mit Daten und wissenschaftlichen Erkenntnissen. Abschließend, am Ende deiner Ausgabe, gebe deine geschätzte Infektionsrate für den Zeitraum und das Bundesland, umschlossen von eckigen Klammern zurück d.h. z.B. Infektionsrate: [0.3].\n"

    data_knowledge = f"Die Infektionszahlen in {bundesland} im Jahr {jahr} und Monat {monat}. Die Fallzahlen sind {fallzahlen}. Die aktuelle Infektionsrate beträgt {infektionsrate}.\n"

    exploration_knowledge = ("Wir haben bereits folgendes rausgefunden: \n"
                             "- Frauen werden deutlich häufiger geimpft als Männer.\n"
                             "- Tendenziell sind Männer leicht häufiger privat versichert als Frauen.\n"
                             "- Menschen, die an Hypertonie erkrankt sind, werden häufiger geimpft als Menschen mit anderen Vorerkrankungen.\n"
                             "- Die Impfquote ist in der Altersgruppe über 60 am höchsten.\n"
                             "- Innerhalb der Menschen mit Risikogruppe, werden über 60 jährige exponentiell häufiger geimpft als unter 60 jährige. Dies gilt nur linear für Patient mit Astma\n"
                             "- Wir wissen, unter Beachtung der Bevölkerungsdichte, dass die Impfquote in den verschiedenen Bundesländern sehr ähnlich ist.\n"
                             "- Unter Beachtung der Bevölkerungsdichte, nimmt die Impfquote linear von 7,8% auf 4,3% ab. Dabei sind die Bundesländer absteigend wie folgt gerankt: Nordrhein-Westfahlen, Niedersachsen, Hessen, Baden-Württemberg, Bayern, Bremen, Sachsen-Anhalt, Schleswig-Holstein, Rheinland-Pfalz, Thüringen, Hamburg, Brandenburg, Berlin, Mecklenburg-Vorpommern, Sachsen, Saarland.\n"
                             )

    return general_prompt + data_knowledge + exploration_knowledge


In [55]:
generate_prompt("NRW", "2022", "Januar", "1000", "0.5")

'Wir führen eine Impfkampagne durch, um die Infektionsrate in NRW im Jahr 2022 und Monat Januar zu senken. Analysiere die bereitgestellten Daten zur aktuellen Impfquote, Infektionsrate und weiteren relevanten Faktoren. Berücksichtige dabei dein Wissen über die Region, einschließlich Demografie, Gesundheitsinfrastruktur und bisheriger Impfeffekte. Schätze ein, wie sich die Impfkampagne voraussichtlich auf die Infektionsrate auswirken wird, und begründe deine Einschätzung mit Daten und wissenschaftlichen Erkenntnissen. Abschließend, am Ende deiner Ausgabe, gebe deine geschätzte Infektionsrate für den Zeitraum und das Bundesland, umschlossen von eckigen Klammern zurück d.h. z.B. Infektionsrate: [0.3].\nDie Infektionszahlen in NRW im Jahr 2022 und Monat Januar. Die Fallzahlen sind 1000. Die aktuelle Infektionsrate beträgt 0.5.\nWir haben bereits folgendes rausgefunden: \n- Frauen werden deutlich häufiger geimpft als Männer.\n- Tendenziell sind Männer leicht häufiger privat versichert als F

In [56]:
cache = Cache(columns=[
    "bundesland", "jahr", "monat", "fallzahlen", "infektionsrate", "prompt", "output", "parsed_value"
])

In [57]:
df = pd.read_csv("merged.csv", index_col=0, header=0)
df

,specialization,Year,CalendarWeek,absolute,extrapolated,Region,InfluenzaCases,Population,InfluenzaCasesPerCapita,VaccinationsPerCapita
0,1,2024,31,1,44,Baden-Württemberg,16,11230740,0.000001,0.000004
1,1,2024,32,4,180,Baden-Württemberg,13,11230740,0.000001,0.000016
2,1,2024,33,3,135,Baden-Württemberg,17,11230740,0.000002,0.000012
3,1,2024,36,1,45,Baden-Württemberg,19,11230740,0.000002,0.000004
4,1,2024,37,66,2952,Baden-Württemberg,19,11230740,0.000002,0.000263
...,...,...,...,...,...,...,...,...,...,...
916,3,2025,1,23,1104,Thüringen,139,2114870,0.000066,0.000522
917,3,2025,2,55,2857,Thüringen,486,2114870,0.000230,0.001351
918,3,2025,3,41,2132,Thüringen,871,2114870,0.000412,0.001008
919,3,2025,4,26,1352,Thüringen,1893,2114870,0.000895,0.000639


In [58]:
df['Date'] = pd.to_datetime(df['Year'].astype(str) + df['CalendarWeek'].astype(str) + '1', format='%Y%W%w')
df['Month'] = df['Date'].dt.month
df = df.drop(columns=['Date'])

In [59]:
dff = df.groupby(['Year', 'Month', 'Region']).sum().reset_index()

In [68]:
def send_row_to_model(row):
    previous_rate = cache.get_from_cache(row['Region'], row['Year'], row['Month'] - 1)
    if previous_rate is None:
        previous_rate = 0

    prompt = generate_prompt(row['Region'], row['Year'], row['Month'], row['Cases'], row['InfluenzaCasesPerCapita'], previous_rate)
    cache_list = [row['Region'], row['Year'], row['Month'], row['Cases'], row['InfluenzaCasesPerCapita'], prompt]

    model_output = lm_emulator(prompt)
    cache_list.append(model_output)

    parsed_value = parse_lm_output(model_output)
    cache_list.append(parsed_value)

    cache.add_to_cache(*cache_list)
    return parsed_value

In [69]:
send_row_to_model(dff.iloc[0])

KeyboardInterrupt: 

In [61]:
dff['predicted_value'] = df.map(lambda x: send_row_to_model(x))

TypeError: 'int' object is not subscriptable